<a href="https://colab.research.google.com/github/cchummer/ml-dl-scratch/blob/main/collab_fast_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd

ratings_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ML+DL/movielens_ratings.csv')
# Grab movie names from other csv
names_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ML+DL/movielens_movies.csv')
# Merge into ratings df
ratings_df = ratings_df.merge(names_df, on='movieId')
ratings_df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [4]:
from fastai.collab import *
from fastai.tabular.all import *

In [21]:
dls = CollabDataLoaders.from_df(ratings_df, item_name='title', bs=64)
dls.show_batch()

,userId,title,rating
0,555,Coneheads (1993),2.0
1,412,North by Northwest (1959),5.0
2,317,L.A. Confidential (1997),4.0
3,380,"Simpsons Movie, The (2007)",4.0
4,246,Hero (Ying xiong) (2002),5.0
5,469,Hear My Song (1991),3.0
6,334,"Femme Nikita, La (Nikita) (1990)",3.0
7,80,"Matrix Revolutions, The (2003)",4.5
8,62,We're the Millers (2013),3.5
9,551,Children of Men (2006),3.0


In [ ]:
dls.classes

In [8]:
n_users  = len(dls.classes['userId'])
n_movies = len(dls.classes['title'])
n_factors = 5

user_factors = torch.randn(n_users, n_factors)
movie_factors = torch.randn(n_movies, n_factors)

print(n_users)
print(n_movies)

611
9720


In [9]:
class DotProductBias(Module):
  def __init__(self, n_users, n_movies, n_factors, y_range=(0,5.5)):
    self.user_factors = Embedding(n_users, n_factors)
    self.user_bias = Embedding(n_users, 1)
    self.movie_factors = Embedding(n_movies, n_factors)
    self.movie_bias = Embedding(n_movies, 1)
    self.y_range = y_range

  def forward(self, x):
    users = self.user_factors(x[:,0])
    movies = self.movie_factors(x[:,1])
    res = (users * movies).sum(dim=1, keepdim=True)
    res += self.user_bias(x[:,0]) + self.movie_bias(x[:,1])
    return sigmoid_range(res, *self.y_range)

In [10]:
model = DotProductBias(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.717983,0.825821,00:25
1,0.464733,0.773595,00:28
2,0.371748,0.751586,00:27
3,0.294628,0.749191,00:25
4,0.258961,0.745445,00:25


In [11]:
movie_bias = learn.model.movie_bias.weight.squeeze()


idxs = movie_bias.argsort(descending=True)[:5]
[dls.classes['title'][i] for i in idxs]



['Shawshank Redemption, The (1994)',
 'Forrest Gump (1994)',
 'Lord of the Rings: The Return of the King, The (2003)',
 'Good Will Hunting (1997)',
 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)']

In [22]:
def test_model_preds(learner, test_dl):

  learner.model.eval()
  for batch in test_dl:

    #print(batch)
    #input()

    userid = batch[0][:, 0]
    itemid = batch[0][:, 1]
    ratings = batch[1]

    x = torch.stack([userid, itemid], dim=1)
    preds = learner.model(x)

    for pred, actual in zip(preds, ratings):
      print(f"Predicted: {pred}, Actual: {actual}")

In [1]:
test_model_preds(learn, dls.valid)

NameError: name 'test_model_preds' is not defined